In [1]:
import glob
import os

import codecs
import pandas as pd
import itertools

In [2]:
os.chdir("./hospital-report")

In [5]:
pd.set_option('display.max_columns', 40)

In [15]:
# score = pd.read_csv('調査対象病院.csv', encoding="utf-8_sig")
score = pd.read_csv('scores/調査対象病院.csv', encoding="shift_jis")

In [16]:
score["区"]=score['住所'].str.split('区', expand=True)[0]+"区"
score.loc[score["総病床数"]>=500, '規模']='大病院'
score.loc[score["総病床数"]<=499, '規模']='中病院'
score.loc[score["総病床数"]<100, '規模']='小病院'
score.to_csv('scores/調査対象病院.csv',  index=False)

## 4つの原則に基づいたスコアの集計

- 全体の集計
- 区毎の集計
- 病床数毎の集計

In [9]:
score.describe().to_csv('scores/全病院.csv')

In [7]:
wards = ['中央区', '千代田区', '台東区', '文京区', '港区']
for w in wards:
    name = f"scores/{w}.csv"
    score[score['区']==w].describe().to_csv(name)

In [8]:
scales = ['小病院', '中病院', '大病院']
for s in scales:
    name = f"scores/{s}.csv"
    score[score['規模']==s].describe().to_csv(name)

## JIS X 8341-3:2016 の未達成項目の分析

- 病院毎の未達成項目数を項目別にカウント
- 病院毎の未達成項目の種類を項目別にカウント

In [3]:
# 達成基準集計用のテーブル作成
df = pd.read_csv('./未達成基準まとめ.csv')
df.fillna(0, inplace=True)

In [154]:
def split_problem(df):
    ls = []
    for i in df[df['種別']=="問題あり"]['JIS']:
        ls.append(i.split(','))
    return list(itertools.chain.from_iterable(ls))

paths = glob.glob('./unachieved-items/*')
for path in paths:
    report = pd.read_csv(path, encoding="shift_jis")
    problems = split_problem(report)
    name = path.split('/')[-1][:-4]
    for problem in problems:
        problem =  problem[3:].replace(' ', '')
#         print(name, problem)
        df.loc[(df['病院名']==name), problem] = list(df[df['病院名']==name][problem])[0]+1

,病院名,1.1.1,1.2.1,1.2.2,1.2.3,1.2.4,1.2.5,1.3.1,1.3.2,1.3.3,1.4.1,1.4.2,1.4.3,1.4.4,1.4.5,2.1.1,2.1.2,2.2.1,2.2.2,2.3.1,2.4.1,2.4.2,2.4.3,2.4.4,2.4.5,2.4.6,2.4.7,3.1.1,3.1.2,3.2.1,3.2.2,3.2.3,3.2.4,3.3.1,3.3.2,3.3.3,3.3.4,4.1.1,4.1.2
0,一般財団法人慈愛病院,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,公益社団法人東京都教職員互助会三楽病院,312.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,249.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,3.0
2,公益財団法人佐々木研究所附属杏雲堂病院,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
3,公益財団法人心臓血管研究所付属病院,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
4,北里大学北里研究所病院,42.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
5,医療法人社団健育会石川島記念病院,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,医療法人社団全仁会上野病院,54.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,258.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,48.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,医療法人社団友仁会赤坂見附前田病院,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,医療法人社団哺育会浅草病院,27.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0
9,医療法人社団大坪会小石川東京病院,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [58]:
output = {}

for c in df.columns[1:]:
    has_problem = df[c]>0
    has_problem.sum()
    output[c] = has_problem.sum()

df_count = pd.DataFrame.from_dict(output, orient='index')
df_count.rename(columns={0: "count"}, inplace=True)
df_count['ratio'] = df_count["count"] / 43
df_count.to_csv('counts/count.csv')

In [63]:
# df_count.sort_values('ratio', ascending=False)

In [152]:
# df.to_csv('未達成基準まとめ.csv', index=False)